<a href="https://colab.research.google.com/github/vermaakarsh/Upgrad/blob/master/EDA_on_StackOverflow_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from collections import Counter

import math
import random

import requests, io

#### Importing the Data

In [2]:
response = requests.get('https://antrikshassignment.s3.amazonaws.com/Questions.csv')
file_object = io.StringIO(response.content.decode('latin-1'))
dfQ = pd.read_csv(file_object)

In [3]:
response = requests.get('https://antrikshassignment.s3.amazonaws.com/Answers.csv')
file_object = io.StringIO(response.content.decode('latin-1'))
dfA = pd.read_csv(file_object)

In [4]:
response = requests.get('https://antrikshassignment.s3.amazonaws.com/Tags.csv')
file_object = io.StringIO(response.content.decode('latin-1'))
dfT = pd.read_csv(file_object)

In [5]:
response = requests.get('https://antrikshassignment.s3.amazonaws.com/Users.csv')
file_object = io.StringIO(response.content.decode('latin-1'))
dfU = pd.read_csv(file_object)

#### Data Cleaning for Questions file

In [6]:
dfQ.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [7]:
# Delete irrelevent columns from Question file
dfQ.drop(["Body"] , axis= 1, inplace= True)
dfQ.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...


#### Data Cleaning for Answers file

In [8]:
dfA.head()

,Id,OwnerUserId,CreationDate,ParentId,Score,Body
0,92,61.0,2008-08-01T14:45:37Z,90,13,"<p><a href=""http://svnbook.red-bean.com/"">Vers..."
1,124,26.0,2008-08-01T16:09:47Z,80,12,<p>I wound up using this. It is a kind of a ha...
2,199,50.0,2008-08-01T19:36:46Z,180,1,<p>I've read somewhere the human eye can't dis...
3,269,91.0,2008-08-01T23:49:57Z,260,4,"<p>Yes, I thought about that, but I soon figur..."
4,307,49.0,2008-08-02T01:49:46Z,260,28,"<p><a href=""http://www.codeproject.com/Article..."


In [9]:
# Delete irrelevent columns from Answer file.
dfA.drop(["Body"] , axis= 1, inplace= True)
dfA.head()

,Id,OwnerUserId,CreationDate,ParentId,Score
0,92,61.0,2008-08-01T14:45:37Z,90,13
1,124,26.0,2008-08-01T16:09:47Z,80,12
2,199,50.0,2008-08-01T19:36:46Z,180,1
3,269,91.0,2008-08-01T23:49:57Z,260,4
4,307,49.0,2008-08-02T01:49:46Z,260,28


#### Merging ParentId in dfA to Id in dfQ

In [10]:
dfQA = pd.merge(dfQ, dfA, how='right', left_on='Id', right_on= 'ParentId')
dfQA.head()

,Id_x,OwnerUserId_x,CreationDate_x,ClosedDate,Score_x,Title,Id_y,OwnerUserId_y,CreationDate_y,ParentId,Score_y
0,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,92,61.0,2008-08-01T14:45:37Z,90,13
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,202317,20709.0,2008-10-14T18:41:45Z,90,2
2,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,1466832,16012.0,2009-09-23T15:40:46Z,90,19
3,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,124,26.0,2008-08-01T16:09:47Z,80,12
4,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,10008,1109.0,2008-08-13T16:09:09Z,80,6


In [11]:
# Let's see what we can make out from the datatypes
# Printing the Data types of Question-Answers file.
dfQA.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2014516 entries, 0 to 2014515
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   Id_x            int64  
 1   OwnerUserId_x   float64
 2   CreationDate_x  object 
 3   ClosedDate      object 
 4   Score_x         int64  
 5   Title           object 
 6   Id_y            int64  
 7   OwnerUserId_y   float64
 8   CreationDate_y  object 
 9   ParentId        int64  
 10  Score_y         int64  
dtypes: float64(2), int64(5), object(4)
memory usage: 184.4+ MB


In [12]:
dfQA = dfQA.dropna(subset = ['OwnerUserId_x', 'OwnerUserId_y'])

# Id_x, OwnerUserId_x, Id_y, OwnerUserId_y, ParentId need to be set to string since they are nominal categorical datatypes
dfQA[['Id_x', 'OwnerUserId_x', 'OwnerUserId_y', 'ParentId', 'Id_y']] = dfQA[['Id_x', 'OwnerUserId_x', 'OwnerUserId_y', 'ParentId', 'Id_y']].astype('int64').astype(str)

dfQA.head()

,Id_x,OwnerUserId_x,CreationDate_x,ClosedDate,Score_x,Title,Id_y,OwnerUserId_y,CreationDate_y,ParentId,Score_y
0,90,58,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,92,61,2008-08-01T14:45:37Z,90,13
1,90,58,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,202317,20709,2008-10-14T18:41:45Z,90,2
2,90,58,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,1466832,16012,2009-09-23T15:40:46Z,90,19
3,80,26,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,124,26,2008-08-01T16:09:47Z,80,12
4,80,26,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,10008,1109,2008-08-13T16:09:09Z,80,6


#### Data Cleaning for Tags file

In [13]:
dfT.head()

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn


In [14]:
# Print the Data types ofTag file
dfT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3750994 entries, 0 to 3750993
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   Id      int64 
 1   Tag     object
dtypes: int64(1), object(1)
memory usage: 57.2+ MB


In [15]:
# Making sure all the datypes in dfT are in string format
dfT['Id'] = dfT.round(0).astype(str)
dfT['Tag'] = dfT['Tag'].astype(str)

dfT_tags_grouped = dfT.groupby(['Id'])['Tag'].apply(','.join).reset_index() # Grouping all the tags together

#### Merging the dfQA (Questions and Answers file) into dfT (tag file)

In [16]:
dfQAT = pd.merge(dfQA, dfT_tags_grouped, how='inner', left_on='Id_x', right_on= 'Id')

#### Data Cleaning for Users file

In [17]:
dfU.head()

,Unnamed: 0,id,display_name,creation_date,last_access_date,reputation,up_votes,down_votes,views
0,0,26,Shawn,2008-08-01 12:18:14.52 UTC,2018-04-11 04:40:35.66 UTC,11820,565,67,3547
1,1,26,Shawn,2008-08-01 12:18:14.52 UTC,2018-04-11 04:40:35.66 UTC,11820,565,67,3547
2,2,26,Shawn,2008-08-01 12:18:14.52 UTC,2018-04-11 04:40:35.66 UTC,11820,565,67,3547
3,3,26,Shawn,2008-08-01 12:18:14.52 UTC,2018-04-11 04:40:35.66 UTC,11820,565,67,3547
4,4,26,Shawn,2008-08-01 12:18:14.52 UTC,2018-04-11 04:40:35.66 UTC,11820,565,67,3547


In [18]:
# Converting required datatypes into proper format
dfU.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1990480 entries, 0 to 1990479
Data columns (total 9 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   Unnamed: 0        int64 
 1   id                int64 
 2   display_name      object
 3   creation_date     object
 4   last_access_date  object
 5   reputation        int64 
 6   up_votes          int64 
 7   down_votes        int64 
 8   views             int64 
dtypes: int64(6), object(3)
memory usage: 136.7+ MB


In [19]:
dfU = dfU.drop(columns = ['Unnamed: 0', 'display_name', 'creation_date','last_access_date'])
dfU['id'] = dfU['id'].astype(str)
dfU = dfU.drop_duplicates()
dfU.head()

,id,reputation,up_votes,down_votes,views
0,26,11820,565,67,3547
27,1109,116072,457,136,6905
85,364174,325,25,1,60
87,61,22779,1321,219,1731
97,20709,978,73,2,96


#### Merging the dfQAT (Questions, Answers, Tag file) into dfU (Users file)

In [20]:
dfQATU = pd.merge(dfQAT, dfU, how='left', left_on='OwnerUserId_y', right_on= 'id')

# Let's begin the analysis!!!

#### Which all Questions had the highest scores

In [21]:
top_Questions =list(zip(dfQ["Id"], dfQ["Score"], dfQ["Title"]))
top_Questions.sort(key=lambda x: x[1],reverse=True)
for id,s,t in top_Questions[:5]:
    print("Question id:",id)
    print("Score :",s)
    print("Question Title\t:",t,'\n')

# Since there is no need of the 'Title' column, we might as well drop it
dfQ.drop(["Title"] , axis= 1, inplace= True)

Question id: 348170
Score : 5190
Question Title	: How to undo 'git add' before commit? 

Question id: 40480
Score : 3613
Question Title	: Is Java "pass-by-reference" or "pass-by-value"? 

Question id: 406230
Score : 2537
Question Title	: Regular expression to match line that doesn't contain a word? 

Question id: 520650
Score : 2399
Question Title	: Make an existing Git branch track a remote branch? 

Question id: 2669690
Score : 2363
Question Title	: Why does Google prepend while(1); to their JSON responses? 



In [22]:
inp0 = dfQATU
inp0.head()

,Id_x,OwnerUserId_x,CreationDate_x,ClosedDate,Score_x,Title,Id_y,OwnerUserId_y,CreationDate_y,ParentId,Score_y,Id,Tag,id,reputation,up_votes,down_votes,views
0,90,58,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,92,61,2008-08-01T14:45:37Z,90,13,90,"svn,tortoisesvn,branch,branching-and-merging",61,22779.0,1321.0,219.0,1731.0
1,90,58,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,202317,20709,2008-10-14T18:41:45Z,90,2,90,"svn,tortoisesvn,branch,branching-and-merging",20709,978.0,73.0,2.0,96.0
2,90,58,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,1466832,16012,2009-09-23T15:40:46Z,90,19,90,"svn,tortoisesvn,branch,branching-and-merging",16012,19888.0,484.0,30.0,1823.0
3,80,26,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,124,26,2008-08-01T16:09:47Z,80,12,80,"flex,actionscript-3,air",26,11820.0,565.0,67.0,3547.0
4,80,26,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,10008,1109,2008-08-13T16:09:09Z,80,6,80,"flex,actionscript-3,air",1109,116072.0,457.0,136.0,6905.0


#### Which tag has the average maximum and minimum time to get the answer?

In [23]:
# Extract the Creation date from CreationDate_x column.
inp0['CreationDate_x']= inp0['CreationDate_x'].apply(lambda x: x.split("T")[0])

# Extract the Creation date from CreationDate_y column.
inp0['CreationDate_y']= inp0['CreationDate_y'].apply(lambda x: x.split("T")[0])

# Convert the Creation date column into date type.
inp0['CreationDate_x'] = pd.to_datetime(inp0['CreationDate_x'])
inp0['CreationDate_y'] = pd.to_datetime(inp0['CreationDate_y'])

In [24]:
# Calculating the time difference between creation date of question and creation date of answer.
# Followed by convertion into integer type

inp0['time_diff_x'] = inp0['CreationDate_y'] - inp0['CreationDate_x']
inp0["time_diff_x"]= inp0["time_diff_x"].astype(str) 
inp0['time_diff_x']= inp0['time_diff_x'].apply(lambda x: x.split(" ")[0])
inp0["time_diff_x"]= inp0["time_diff_x"].astype(int)
inp0.head()

,Id_x,OwnerUserId_x,CreationDate_x,ClosedDate,Score_x,Title,Id_y,OwnerUserId_y,CreationDate_y,ParentId,Score_y,Id,Tag,id,reputation,up_votes,down_votes,views,time_diff_x
0,90,58,2008-08-01,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,92,61,2008-08-01,90,13,90,"svn,tortoisesvn,branch,branching-and-merging",61,22779.0,1321.0,219.0,1731.0,0
1,90,58,2008-08-01,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,202317,20709,2008-10-14,90,2,90,"svn,tortoisesvn,branch,branching-and-merging",20709,978.0,73.0,2.0,96.0,74
2,90,58,2008-08-01,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,1466832,16012,2009-09-23,90,19,90,"svn,tortoisesvn,branch,branching-and-merging",16012,19888.0,484.0,30.0,1823.0,418
3,80,26,2008-08-01,NaN,26,SQLStatement.execute() - multiple queries in o...,124,26,2008-08-01,80,12,80,"flex,actionscript-3,air",26,11820.0,565.0,67.0,3547.0,0
4,80,26,2008-08-01,NaN,26,SQLStatement.execute() - multiple queries in o...,10008,1109,2008-08-13,80,6,80,"flex,actionscript-3,air",1109,116072.0,457.0,136.0,6905.0,12


In [ ]:
# To avoid any repition of tags, let us consider the minimum time required to answer any particular answer
inp1 = inp0.loc[inp0.groupby("Id_x")["time_diff_x"].idxmin()]

In [ ]:
tag_time = inp1[['Tag', 'time_diff_x']].drop_duplicates().reset_index(drop=True)
tag_time.head()

In [ ]:
tagTime = {}
tag_time = tag_time.dropna().reset_index()

for i in tqdm_notebook(range(len(tag_time['Tag']))):
    for tag in tag_time['Tag'][i].split(','):
        try:
            tagTime[tag].append(tag_time['time_diff_x'][i])
        except Exception:
            tagTime[tag] = [tag_time['time_diff_x'][i]]

In [ ]:
tagTimeAvg = {}
for key in list(tagTime.keys()):
    tagTimeAvg[key] = np.average(tagTime[key])

#### What would be the expected time for your question to be answered given the tag is 'excel'

In [ ]:
print('My question is expected to be answered within ', tagTimeAvg['excel'], ' days')

In [ ]:
tagTimeAvg = pd.DataFrame([list(tagTimeAvg.keys()), list(tagTimeAvg.values())])

tagTimeAvg = tagTimeAvg.transpose()
tagTimeAvg.columns = ['Tag', 'time_diff_x']
tagTimeAvg = tagTimeAvg.sort_values('time_diff_x')

In [ ]:
# Top 20 tags with highest average time to answer

highest_tag_time= tagTimeAvg.tail(20)
highest_tag_time.plot.barh(x='Tag', y='time_diff_x', rot=0)

#### Corresponding expected answer time for data science tags

In [ ]:
data_science = ['python', 'r', 'matlab', 'sas', 'excel', 'sql']
x = []
y = []

for tag in data_science:
    index = list(tagTimeAvg['Tag']).index(tag)
    x.append(tag)
    y.append(list(tagTimeAvg['time_diff_x'])[index])
    
print(x)
print(y)

In [ ]:
DS_time_diff_df = pd.DataFrame(list(zip(x, y)), columns =['DS Tag', 'time_diff'])
DS_time_diff_df.plot.bar(x='DS Tag', y='time_diff', rot=0)

In [ ]:
DS_time_diff_df

#### Should you have more tags for a question to be anwered quickly?

In [ ]:
tag_time['tag_count'] = [len(tags.split(',')) for tags in tag_time['Tag']]

tag_time_grouped = tag_time.groupby('tag_count').mean()
tag_time_grouped.reset_index(inplace=True)
sns.barplot(x='tag_count', y='time_diff_x', data = tag_time_grouped)

#### What are the most popular tags?

In [ ]:
tags = list(inp1['Tag'].unique())
all_tags = list(dfT['Tag'].unique())
print('Number of tags are ', len(all_tags))

In [ ]:
count_tags = Counter(list(dfT['Tag'])) # Using dfT (Tags Dataframe) to find the overall frequency of each tag
tags = pd.DataFrame([list(count_tags.keys()), list(count_tags.values())])
tags = tags.transpose()
tags.columns = ['tag', 'count']
tags = tags.sort_values(by='count', ascending=False)

#### Creating a bar chart of the top 20 most popular tags

In [ ]:
Tags_df= tags.head(20)
Tags_df.plot.barh(x='tag', y='count', rot=0)

#### How popular are some of the data science tags?

In [ ]:
data_science = ['python', 'r', 'matlab', 'sas', 'excel', 'sql']
x = []
y = []

for tag in data_science:
    index = list(tags['tag']).index(tag)
    x.append(tag)
    y.append(list(tags['count'])[index])
    
print(x)
print(y)

In [ ]:
DS_Tags_df = pd.DataFrame(list(zip(x, y)), columns =['Tag', 'Values'])
DS_Tags_df.plot.bar(x='Tag', y='Values', rot=0)

#### What are the most popular tags amongst reputed users?

In [ ]:
# First let us set a threshold to classify any user as reputed or not
sns.boxplot(inp0['reputation']) # Creating a boxplot of the reputation of all users

In [ ]:
inp0.reputation.quantile([0.01, 0.1, 0.2, 0.3, 0.5, 0.7, 0.9, 0.95, 0.99, 0.999, 1])

In [ ]:
inp0[inp0['reputation'] > inp0['reputation'].quantile(0.99)].shape

In [ ]:
# Hence we will be filtering out within the top 19044 users
# So we shall consider those users whose reputation lie in the top 99 percentile

inp1 = inp0[inp0['reputation'] > inp0['reputation'].quantile(0.99)]

reputed_users_tags = inp1['Tag'].str.cat(sep=',')
reputed_users_tags_dict = Counter(reputed_users_tags.split(','))

reputed_users_tags_df = pd.DataFrame([list(reputed_users_tags_dict.keys()), list(reputed_users_tags_dict.values())])
reputed_users_tags_df = reputed_users_tags_df.transpose()
reputed_users_tags_df.columns = ['tag', 'count']
reputed_users_tags_df = reputed_users_tags_df.sort_values(by='count', ascending=False)
reputed_users_tags_df.head()

#### What are the most popular tags amongst the closed questions

In [ ]:
inp1 = inp0.dropna(subset = ['ClosedDate'])
Questions_closed_date_tags = inp1['Tag'].str.cat(sep=',')
Questions_closed_date_tags_dict = Counter(Questions_closed_date_tags.split(','))

In [ ]:
Questions_closed_date_tags_df = pd.DataFrame([list(Questions_closed_date_tags_dict.keys()), list(Questions_closed_date_tags_dict.values())])
Questions_closed_date_tags_df = Questions_closed_date_tags_df.transpose()
Questions_closed_date_tags_df.columns = ['tag', 'count']
Questions_closed_date_tags_df = Questions_closed_date_tags_df.sort_values(by='count', ascending=False)
Questions_closed_date_tags_df.head()

#### Which tags have the highest scores?

In [ ]:
tag_scores = inp0[['Id_x', 'Tag', 'Score_x']].drop_duplicates().reset_index(drop=True)
tag_scores = tag_scores.dropna(axis = 0, how = 'any', inplace = False).reset_index(drop=True) # Removing any nan values
tag_scores.head()

In [ ]:
tagScore = {}
for i in tqdm_notebook(range(len(tag_scores['Tag']))):
    
    for tag in tag_scores['Tag'][i].split(','):
        try:
            tagScore[tag].append(tag_scores['Score_x'][i])
        except Exception:
            tagScore[tag] = [tag_scores['Score_x'][i]]

In [ ]:
## Determining the average score for each tag

tagScoreAvg = {}
for key in list(tagScore.keys()):
    tagScoreAvg[key] = np.average(tagScore[key])

In [ ]:
# Using pandas dataframe for sorting

tagScoreAvg = pd.DataFrame([list(tagScoreAvg.keys()), list(tagScoreAvg.values())])
tagScoreAvg = tagScoreAvg.transpose()
tagScoreAvg.columns = ['Tag', 'Score_x']
tagScoreAvg = tagScoreAvg.sort_values('Score_x')

In [ ]:
# Top 20 tags with highest average scores

high_tag= tagScoreAvg.tail(20)
high_tag.plot.barh(x='Tag', y='Score_x', rot=0)

In [ ]:
# Top 20 tags with lowest average scores

low_tag= tagScoreAvg.head(20)
low_tag.plot.barh(x='Tag', y='Score_x', rot=0)

In [ ]:
import random

random.seed(0)

# We have selected 10 random tags amongst the top 50 most popular tags and appended it with few data science tags which we can
# use for comparison later

n = 10
random_tags = []

i = 0
while i<n:
    random_tag = tags['tag'][random.randint(0, 50)]
    if random_tag not in data_science:
        random_tags.append(random_tag)
        i = i+1
    
top_scoring_tags = np.append(data_science, random_tags)

In [ ]:
top_scoring_tags

#### Let us try to find those users who answer questions for any particular tag most frequently

In [ ]:
dfQATU_owners_alltags = inp0.groupby(['OwnerUserId_y'])['Tag'].apply(','.join).reset_index()
dfQATU_owners_alltags = dfQATU_owners_alltags.reset_index()
dfQATU_owners_alltags.head()

In [ ]:
dfQATU_owners_tagcount = {}
dfQATU_owners_tagcount = {tag_user : Counter(dfQATU_owners_alltags['Tag'][k].split(',')) for k, tag_user in enumerate(dfQATU_owners_alltags['OwnerUserId_y'])}

In [ ]:
#tag_top_users = {tag:[] for tag in top_scoring_tags}
tag_all_users = {tag:[] for tag in top_scoring_tags}

for user in tqdm_notebook(dfQATU_owners_tagcount.keys()):
    common_tags = dfQATU_owners_tagcount[user].keys() & top_scoring_tags
    for tag in common_tags:
        tag_all_users[tag].append(user)
    

#### Let's sort it and get the top 10 users who have answered the most questions for any particular tag 

In [ ]:
most_questions_answered_users_dict = {}

for tag in top_scoring_tags:
    
    most_questions_answered_users = tag_all_users[tag]
    most_questions_answered_count = []

    for user in tag_all_users[tag]:
        most_questions_answered_count.append(dfQATU_owners_tagcount[user][tag])

    tag_all_users_df = {'Users': most_questions_answered_users, 'Answered': most_questions_answered_count}
    tag_all_users_df = pd.DataFrame(tag_all_users_df)
    tag_all_users_df = tag_all_users_df.sort_values('Answered', ascending = False)

    most_questions_answered_users_dict[tag] = tag_all_users_df.head(10)

In [ ]:
top_scoring_tags

In [ ]:
# So suppose you we want to find the top 10 users who have answered most questions in sql
most_questions_answered_users_dict['sql']

#### Trend of some data science and non - data science tags

In [ ]:
dfQATU_trend = inp0[['CreationDate_x', 'Tag']]

dfQATU_trend = dfQATU_trend.sort_values(by='CreationDate_x')
dfQATU_trend_day = dfQATU_trend.copy()

In [ ]:
dfQATU_trend['CreationDate_x'] = dfQATU_trend['CreationDate_x'].dt.to_period('M') # Extracting only the month and year
dfQATU_trend['CreationDate_x'] = dfQATU_trend['CreationDate_x'].astype(str)

dateline = dfQATU_trend['CreationDate_x'].unique()

In [ ]:
dfQATU_trend = dfQATU_trend.groupby(['CreationDate_x'])['Tag'].apply(','.join).reset_index()
dfQATU_trend.head()

In [ ]:
print('Our time series analysis will be performed on the following tags:')
print(top_scoring_tags)

In [ ]:
top_scoring_tags_dict = {tag:[] for tag in top_scoring_tags}

# Counting the number of times, a particular keyword was tagged per month

for k, month in enumerate(dfQATU_trend['CreationDate_x']):
    tag_counts = Counter(dfQATU_trend['Tag'][k].split(','))
    for top_tag in top_scoring_tags:
        try:
            top_scoring_tags_dict[top_tag].append(tag_counts[top_tag])
        except Exception:
            top_scoring_tags_dict[top_tag].append(0)

In [ ]:
# Final Plotting

plt.figure(figsize=(20,12)) 
for tag in top_scoring_tags_dict:
    plt.plot(top_scoring_tags_dict[tag], label = tag)
    
plt.ylabel('Number of tags per month')
plt.xlabel('Year - Month')
plt.title('Trend of tag popularity over the years')
plt.xticks(np.arange(len(dateline)), dateline, rotation=90)
plt.legend(loc = 'best')
plt.show()

#### Let us do a similar plot for analysing only the yearly growth

In [ ]:
dfQATU_trend['CreationDate_year'] = dfQATU_trend['CreationDate_x'].apply(lambda x: x.split("-")[0]) # Extracting only the year
yearline = dfQATU_trend['CreationDate_year'].unique()

In [ ]:
dfQATU_year_trend = dfQATU_trend.groupby(['CreationDate_year'])['Tag'].apply(','.join).reset_index()
dfQATU_year_trend.head()

In [ ]:
top_scoring_tags_yearly = {tag:[] for tag in top_scoring_tags}

# Counting the number of times, a particular keyword was tagged per year

for k, year in enumerate(dfQATU_year_trend['CreationDate_year']):
    tag_counts = Counter(dfQATU_year_trend['Tag'][k].split(','))
    for top_tag in top_scoring_tags:
        try:
            top_scoring_tags_yearly[top_tag].append(tag_counts[top_tag])
        except Exception:
            top_scoring_tags_yearly[top_tag].append(0)

In [ ]:
# Final Plotting

plt.figure(figsize=(20,12)) 
#for tag in top_scoring_tags_dict:
for tag in top_scoring_tags_dict:
    plt.plot(top_scoring_tags_yearly[tag], label = tag)
    
plt.ylabel('Popularity of the tag')
plt.xlabel('Year')
plt.title('Yearly growth of the popularity of each tag')
plt.xticks(np.arange(len(yearline)), yearline, rotation=90)
plt.legend(loc = 'best')
plt.show()

#### Analysing the Growth Rate of tags over the years

In [ ]:
top_scoring_tags_growth = {}
for tag in top_scoring_tags_yearly.keys():
    slope = []
    for i in range(len(top_scoring_tags_yearly[tag])-1):
        slope.append(top_scoring_tags_yearly[tag][i+1] - top_scoring_tags_yearly[tag][i])
    top_scoring_tags_growth[tag] = slope

In [ ]:
# Final Plotting

plt.figure(figsize=(20,12))
for tag in top_scoring_tags_dict:
    plt.plot(top_scoring_tags_growth[tag], label = tag)
    
plt.ylabel('Growth rate')
plt.xlabel('Year')
plt.title('Growth rate of tag popularity per year')
plt.xticks(np.arange(len(yearline)-1), yearline[1:], rotation=90)
plt.legend(loc = 'best')
plt.show()

#### Comparing the popularity of tags for different months

In [ ]:
dfQATU_trend['CreationDate_month'] = dfQATU_trend['CreationDate_x'].apply(lambda x: x.split("-")[1]) # Extracting only the year
monthline = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

dfQATU_month_trend = dfQATU_trend.groupby(['CreationDate_month'])['Tag'].apply(','.join).reset_index()
dfQATU_month_trend

In [ ]:
top_scoring_tags_months = {tag:[] for tag in top_scoring_tags}

# Counting the number of times, a particular keyword was tagged per month

for k, year in enumerate(dfQATU_month_trend['CreationDate_month']):
    tag_counts = Counter(dfQATU_month_trend['Tag'][k].split(','))
    for top_tag in top_scoring_tags:
        try:
            top_scoring_tags_months[top_tag].append(tag_counts[top_tag])
        except Exception:
            top_scoring_tags_months[top_tag].append(0)

In [ ]:
# Final Plotting

plt.figure(figsize=(12,7)) 
for tag in top_scoring_tags_months:
    plt.plot(top_scoring_tags_months[tag], label = tag)
    
plt.ylabel('Popularity of the tag')
plt.xlabel('Month')
plt.title('Comparing the popularity of tags for each month')
plt.xticks(np.arange(len(monthline)), monthline, rotation=90)
plt.legend(loc = 'upper left')
plt.show()

In [ ]:
# Tags seem to be used slightly less frequently during the summer and winter holidays, don't you think?

In [ ]:
Days_of_Week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dfQATU_trend_day['CreationDate_x'] = dfQATU_trend_day['CreationDate_x'].dt.day_name().astype(str)
dfQATU_day_trend = dfQATU_trend_day.groupby('CreationDate_x')['Tag'].apply(','.join).reindex(Days_of_Week)
dfQATU_day_trend

In [ ]:
top_scoring_tags_day = {tag:[] for tag in top_scoring_tags}

# Counting the number of times, a particular keyword was tagged on a particular day of the week

for k, day in enumerate(dfQATU_day_trend):
    tag_counts = Counter(dfQATU_day_trend[k].split(','))
    for top_tag in top_scoring_tags:
        try:
            top_scoring_tags_day[top_tag].append(tag_counts[top_tag])
        except Exception:
            top_scoring_tags_day[top_tag].append(0)

In [ ]:
# Final Plotting

plt.figure(figsize=(12,7)) 
for tag in top_scoring_tags_day:
    plt.plot(top_scoring_tags_day[tag], label = tag)
    
plt.ylabel('Popularity of the tag')
plt.xlabel('Day of the week')
plt.title('Comparing the popularity of tags on different days of the week')
plt.xticks(np.arange(len(Days_of_Week)), Days_of_Week, rotation=90)
plt.legend(loc = 'best')
plt.show()

In [ ]:
# Looks like people like being more active  during the weekdays